In [1]:
import os
import pandas as pd
from pathlib import Path
from function import load_data
from nav_research import NavResearch

Welcome to use Wind Quant API for Python (WindPy)!

COPYRIGHT (C) 2024 WIND INFORMATION CO., LTD. ALL RIGHTS RESERVED.
IN NO CIRCUMSTANCE SHALL WIND BE RESPONSIBLE FOR ANY DAMAGES OR LOSSES CAUSED BY USING WIND QUANT API FOR Python.


In [2]:
fund_info = load_data("产品代码.xlsx")
fund_info["基金代码"] = fund_info["基金代码"].astype(str)

In [3]:
fund_info

,大类策略,基金代码,基金名称,基准代码,基准名称,最大回撤阈值
0,灵活配置,SGV993,聚鸣涌金8号A期,881001.WI,万得全A,-0.1
1,灵活配置,SGJ508,玄元元丰2号A,881001.WI,万得全A,-0.1
2,灵活配置,SLY951,山楂树五期,881001.WI,万得全A,-0.1
3,灵活配置,SSS087,元品南星A,881001.WI,万得全A,-0.1
4,灵活配置,SLZ483,循远安心十三号A,881001.WI,万得全A,-0.1
...,...,...,...,...,...,...
84,多策略,SVC372,博普CTA股票全市场五号1期,881001.WI,万得全A,-0.1
85,多策略,953832,君享大类资产全天候2号,881001.WI,万得全A,-0.1
86,其他,SSB167,天演天睿,881001.WI,万得全A,-0.1
87,其他,STU233,博普凤祥13号,881001.WI,万得全A,-0.1


In [4]:
files_list_series = pd.Series([i for i in Path("./nav_data").rglob("*") if i.suffix.lower() in {".csv", ".xlsx",".xls"}])
nav_dfs = pd.DataFrame()
for row in fund_info.itertuples(index=False, name=None):
    nav_df_path = files_list_series[files_list_series.apply(lambda x: row[1] in x.stem)]
    nav_df = load_data(nav_df_path.iloc[0])
    nav_df["基金代码"] = row[1]
    nav_df["基金名称"] = row[2]
    nav_df = nav_df[["基金代码","基金名称","日期","单位净值","累计净值"]]
    nav_dfs = pd.concat([nav_dfs, nav_df], ignore_index=True)
nav_dfs.to_csv("nav_dfs.csv",index=False, encoding="utf-8")

In [5]:
nav_dfs

,基金代码,基金名称,日期,单位净值,累计净值
0,SGV993,聚鸣涌金8号A期,2020-01-22,1.0000,1.0000
1,SGV993,聚鸣涌金8号A期,2020-02-05,1.0000,1.0000
2,SGV993,聚鸣涌金8号A期,2020-02-06,1.0000,1.0000
3,SGV993,聚鸣涌金8号A期,2020-02-07,1.0000,1.0000
4,SGV993,聚鸣涌金8号A期,2020-02-10,1.0000,1.0000
...,...,...,...,...,...
20487,SXA265,玖鹏宏远3号,2025-02-05,0.9712,1.0965
20488,SXA265,玖鹏宏远3号,2025-02-06,0.9561,1.0814
20489,SXA265,玖鹏宏远3号,2025-02-11,1.0120,1.1373
20490,SXA265,玖鹏宏远3号,2025-02-18,1.0401,1.1654


In [7]:
def delete_csv_files(directory):
    # 遍历目录及其子目录
    for root, dirs, files in os.walk(directory):
        for file in files:
            # 检查文件是否以.csv 结尾
            if file.endswith(".csv"):
                file_path = os.path.join(root, file)
                try:
                    # 删除文件
                    os.remove(file_path)
                    # print(f"Deleted: {file_path}")
                except Exception as e:
                    print(f"Error deleting {file_path}: {e}")
delete_csv_files("nav_data")

In [8]:
nav_dfs = pd.read_csv("nav_dfs.csv")

In [9]:
# 确保输出目录存在
os.makedirs("nav_data", exist_ok=True)

# 检查日期列的类型，如果不是 datetime 类型，则转换为 datetime 类型
if nav_dfs['日期'].dtype != 'datetime64[ns]':
    nav_dfs['日期'] = pd.to_datetime(nav_dfs['日期'], format='%Y-%m-%d')

# 遍历 nav_dfs 中 "基金代码" 列的唯一值
for fund_code in nav_dfs["基金代码"].unique():
    nav_df = nav_dfs[nav_dfs["基金代码"] == fund_code]
    start_date = nav_df["日期"].min().strftime('%Y%m%d')
    end_date = nav_df["日期"].max().strftime('%Y%m%d')
    fund_name = nav_df["基金名称"].iloc[0]
    file_path = os.path.join("nav_data", f"{fund_code}_{fund_name}_{start_date}_{end_date}.csv")
    nav_df.to_csv(file_path, index=False)

In [16]:
def multi_fund_comparison(tables, fund_name):
    data1 = tables[0]
    data2 = tables[1].head(1)
    data3 = pd.DataFrame()
    data4 = tables[2]
    years = [2019, 2020, 2021, 2022, 2023, 2024, 2025]
    for year in years:
        data3[f"{year}收益"] = data4.loc[
            data4["分年度业绩"] == year, f"{fund_name}_收益"
        ].values
        data3[f"{year}最大回撤"] = data4.loc[
            data4["分年度业绩"] == year, f"{fund_name}_最大回撤"
        ].values
    data = pd.concat([data1, data2, data3], axis=1)
    data.drop(columns=["基准指数", "整体业绩"], inplace=True)
    return data


data = pd.DataFrame()
files_list_series = pd.Series(
    [i for i in Path("./nav_data").rglob("*") if i.suffix.lower() in {".csv", ".xlsx",".xls"}]
)
for row in fund_info.itertuples(index=False, name=None):
    print(row[2])
    nav_df_path = files_list_series[files_list_series.apply(lambda x: row[1] in x.stem)]
    assert len(nav_df_path) == 1, "找到多个文件或者没有文件"
    demo = NavResearch(nav_df_path.item(), row[0], row[2], row[3], row[4], row[5])
    demo.get_data()
    tables = demo.get_analysis_table()
    nav_df = multi_fund_comparison(tables, row[2])
    data = pd.concat([data, nav_df], axis=0)

data.to_excel("report_data.xlsx", index=False)

聚鸣涌金8号A期
玄元元丰2号A
玄元元丰2号A无法推断频率,自动转为周度
山楂树五期
元品南星A
循远安心十三号A
循远安心十三号A无法推断频率,自动转为周度
聚鸣匠传11号
聚鸣匠传11号无法推断频率,自动转为周度
聚鸣章玉价值成长3号A
聚鸣章玉价值成长3号A无法推断频率,自动转为周度
健顺云6号C
健顺云6号C无法推断频率,自动转为周度
翊安投资可转债2号
翊安投资可转债2号无法推断频率,自动转为周度
弘尚资产弘利2号1期
弘尚资产弘利2号1期无法推断频率,自动转为周度
勤辰创赢成长6号1期
勤辰创赢成长6号1期无法推断频率,自动转为周度
勤辰金选森裕弘享1号
勤辰金选森裕弘享1号无法推断频率,自动转为周度
仁布积极进取14号A
格雷长期价值16号B
格雷长期价值16号B无法推断频率,自动转为周度
彤源同裕1期1号
彤源同裕1期1号无法推断频率,自动转为周度
石锋资产笃行7号A期
石锋资产笃行7号A期无法推断频率,自动转为周度
重阳福享1期
源乐晟-晟世10号8期
佳岳实投精选一期
景林精选FOF子基金GJ2期
景林精选FOF子基金GJ2期无法推断频率,自动转为周度
景林景泰丰收GJ2期
景林景泰丰收GJ2期无法推断频率,自动转为周度
景林景泰优选GJ2期
景林景泰优选GJ2期无法推断频率,自动转为周度
沣京公司精选2期
沣京公司精选2期无法推断频率,自动转为周度
睿郡财富11号1期
睿郡财富11号1期无法推断频率,自动转为周度
高毅任昊致福25期A
高毅任昊致福25期A无法推断频率,自动转为周度
盛天国华1号A
旭鑫价值成长7期
旭鑫价值成长7期无法推断频率,自动转为周度
易同精选2号
易同精选2号无法推断频率,自动转为周度
宽远安泰成长1号
宽远安泰成长1号无法推断频率,自动转为周度
合撰价值精选贰号
合撰价值精选贰号无法推断频率,自动转为周度
明达精选V号1期
明达精选V号1期无法推断频率,自动转为周度
东方港湾汉韵一期
东方港湾汉韵一期无法推断频率,自动转为周度
静瑞灵动增长安泰1号
静瑞灵动增长安泰1号无法推断频率,自动转为周度
兴聚投资可换股债券1号
兴聚投资可换股债券1号无法推断频率,自动转为周度
高毅臻选FOF21期A
高毅臻选FOF21期A无法推断频率,自动转为周度
睿郡承非郡享1号
远信中国优质企业逆向策略5号1期
衍复新擎300增强一号